# <center> IBM Data Science Professional Certificate Capstone Project </center>

This notebook will be used for my Capstone Project of IBM Data Science Professional Certificate course powered by Coursera

## <center>The Battle of Neighborhoods</center>

In [159]:
import pandas as pd
import numpy as np

In [160]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Toronto Neighbors Clustering   
#### **Notebook 1 of Final Assignment for Applied Data Science Capstone course**

## **Part 1. Toronto Neighborhoods Data Web Scraping into Dataframe**

Let's start by installing requests and bs4 for scraping the Toronto neighbors table from Wikipedia

In [161]:
!pip install requests
!pip install bs4
!pip install geopy
!pip install folium

Let's import now all the needed libraries and set display options

In [162]:
import requests
from bs4 import BeautifulSoup

import types
from botocore.client import Config
import ibm_boto3
def __iter__(self): return 0

from geopy.geocoders import Nominatim

import folium

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import math

In [163]:
# The code was removed by Watson Studio for sharing.

We will use Beautiful Soup in order to scrap the Toronto Neighbors table from Wikipedia webpage

In [164]:
scrapUrl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # the url of the webpage we want to scrape

response = requests.get(scrapUrl) # we use requests to get the webpage at the set url

soup = BeautifulSoup(response.content, 'html.parser') # we use Beautiful Soup to parse the file got by the requests.get() method 

myTable = soup.find("table") # we save the html code between tag <table> in the myTable variable

tableRows = myTable.find_all("tr") # we save all the table rows code in the tableRows variable

Now that we have the "source" html code for all the rows in the table of Toronto Neighbors stored in the tableRows variable, we use 2 for loops:  
1. the first is used to create a list of lists. Each list contains the values of each Toronto Neighbors table row: if the field Neighbor has a value 'Not assigned', it is skipped as requested in the final assignment instructions; if the field Neighborhood has a value 'Not assigned',, but its relative field 'Neighbors' has a proper value (i.e. it has not 'Not assigned' value), than the field Neighborhood is filled with the same value of its relative field 'Neighbors';  
2. the second is used to check if:  
 * there are two or more redundant rows: in that case all copies of the same rows will be deleted  
 * some rows has the same PostalCode of another row: in that case, their Neighborhood values are all merged in one row and the other rows indexes are stored in a list. Finally this list will be used to delete the redundant values rows from tableRows list 

In [165]:
rowsList = [] # list of lists of table rows values
delList = [] # list to store the indexes of rows to delete

for tr in tableRows:
    td = tr.find_all("td")
    row = [i.text[:-1] for i in td]
    if len(row) != 0 and row[1] != "Not assigned":
        if row[2] == "Not assigned":
            row[2] = row[1]
        rowsList.append(row)

for i, row in enumerate(rowsList):
    for j, row2 in enumerate(rowsList):
        if (i == j) or (i in delList): # if we are checking the same row, or if we are checking a row that is a copy or must be merged 
                                       # with another, let's skip the row
            continue
        elif row2 == row:
            delList.append(j) # if two rows are identical, we will delete (in another for loop) the copy
        elif row2[0] == row[0] and row2[2] != row[2]: # if we have the same PostalCode for two or more rows, we merge them and then (in another 
                                                      # for loop) delete the redundant PostalCode rows
            row[2] = str(row[2]) + ", " + str(row2[2])
            delList.append(j)
        else:
            continue

for j in delList:
    rowsList.pop(j) # let's remove the redundant rows

Perfect. Now the rowsList is clean and has the required format. We can use it to create our Pandas dataframe of Toronto Neighbors as requested in the final assignment

In [166]:
dfToronto = pd.DataFrame(np.array(rowsList), columns=["PostalCode", "Borough", "Neighborhood"])
dfToronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Let's check if we have some missing value

In [167]:
dfToronto.isnull().sum()

PostalCode      0
Borough         0
Neighborhood    0
dtype: int64

Everything seems ok.  
Let's finally check the dimensions of our dataframe

In [168]:
dfToronto.shape

(103, 3)

Ok, currently we have **103 rows and 3 columns** in our dataframe.  
### **This concludes the first part of the first Notebook for final assignment.**

## **Part 2. Adding Latitude and Longitude to Toronto Neighbors Dataframe**

Let's start by building a geocoder object, i.e. an object able to return the latitude and longitude of PostalCode **<-- changed strategy** 

In [169]:
# geocoder = Nominatim(user_agent="Toronto_dc") # I have commented this as the following cell code is extremely inefficient for the purpose

Now let's use our geocoder in order to store in our dataframe the latitude and the longitude of every PostalCode **<-- changed strategy** 

In [170]:
# the following code works, but it is extremely inefficient. This is the only soluton I have found if one doesn't want to use services (geocoders)
# that require api key and that potentially can charge your credit card (as the GoogleV3 one). In order to finsih it takes hours or maybe days.
# I have killed the running after almost 3 hours, when the code was able to grab only the first 5 rows. So I will go on using the supplied .csv
# file with lat. and long. values for Toronto Neighbors dataframe
'''
dictionary = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
dictionary2 = ['0','1','2','3','4','5','6','7','8','9']
latList = [0] * dfToronto.shape[0]
longList = [0] * dfToronto.shape[0]
tempLatList = []
tempLongList = []
for i, pCode in enumerate(dfToronto['PostalCode']):    
    for char1 in dictionary2:
        for char2 in dictionary:
            for char3 in dictionary2:
                finalPart = char1+char2+char3
                postalCode = pCode + ' ' + finalPart
                location = geocoder.geocode({'postalcode': postalCode}, country_codes='ca')
                if location:
                    lat = location.latitude
                    long = location.longitude
                    tempLatList.append(lat)
                    tempLongList.append(long)
                    print(postalCode, " found!")
                if char3 == '9' and char2 == 'Z' and char1 == '9':
                    meanLat = sum(tempLatList)/len(tempLatList)
                    meanLong = sum(tempLongList)/len(tempLongList)
                    latList[i] = meanLat
                    longList[i] = meanLong
                    dfToronto['Latitude'] = latList
                    dfToronto['Longitude'] = longList
                    dfToronto.to_csv('dfToronto.csv')
                    tempLatList.clear()
                    tempLongList.clear()
                    print(i+1, " of ", dfToronto.shape[0], " done!")

print('work completed!')
'''

'\ndictionary = [\'A\',\'B\',\'C\',\'D\',\'E\',\'F\',\'G\',\'H\',\'I\',\'J\',\'K\',\'L\',\'M\',\'N\',\'O\',\'P\',\'Q\',\'R\',\'S\',\'T\',\'U\',\'V\',\'W\',\'X\',\'Y\',\'Z\']\ndictionary2 = [\'0\',\'1\',\'2\',\'3\',\'4\',\'5\',\'6\',\'7\',\'8\',\'9\']\nlatList = [0] * dfToronto.shape[0]\nlongList = [0] * dfToronto.shape[0]\ntempLatList = []\ntempLongList = []\nfor i, pCode in enumerate(dfToronto[\'PostalCode\']):    \n    for char1 in dictionary2:\n        for char2 in dictionary:\n            for char3 in dictionary2:\n                finalPart = char1+char2+char3\n                postalCode = pCode + \' \' + finalPart\n                location = geocoder.geocode({\'postalcode\': postalCode}, country_codes=\'ca\')\n                if location:\n                    lat = location.latitude\n                    long = location.longitude\n                    tempLatList.append(lat)\n                    tempLongList.append(long)\n                    print(postalCode, " found!")\n           

Let's import into a dataframe the .csv file with lat. and long. values

In [171]:
# The code was removed by Watson Studio for sharing.

In [172]:
# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
dfLatLong = pd.read_csv(Geospatial_Coordinates)
dfLatLong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's check dfLatLong shape in order to see we have the same number of rows as dfToronto

In [173]:
dfLatLong.shape[0] == dfToronto.shape[0]

True

Let's sort dfLatLong by "Postal Code" column

In [174]:
dfLatLong.sort_values('Postal Code', inplace=True)
dfLatLong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's sort dfToronto by PostalCode column

In [175]:
dfToronto.sort_values('PostalCode', inplace=True)
dfToronto.reset_index(inplace=True)
dfToronto.head()

,index,PostalCode,Borough,Neighborhood
0,6,M1B,Scarborough,"Malvern, Rouge"
1,12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,22,M1G,Scarborough,Woburn
4,26,M1H,Scarborough,Cedarbrae


Let's check Postal Code values are the same in both dataframes

In [176]:
checkPC = dfToronto['PostalCode'] == dfLatLong['Postal Code']
j = 1
for myBool in checkPC:
    if not myBool:
        j = 0
        print("Warning! Series are not equal")
if j:
    print("SERIES ARE EQUALS!!!")

SERIES ARE EQUALS!!!


Let's drop column "index" from dfToronto and column "Postal Code" from dfLatLong

In [177]:
dfToronto.drop('index', axis=1, inplace=True)
dfLatLong.drop('Postal Code', axis=1, inplace=True)
dfToronto.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [178]:
dfLatLong.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


Let's finally join the two dataframe

In [179]:
dfToronto = pd.concat([dfToronto, dfLatLong], axis=1)
dfToronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### **This concludes the second part of the first Notebook for final assignment.**

## **Part 3. Clustering and analyzing Toronto Neighbors**

Let's plot the Toronto map with its Boroughs and Neighborhoods

In [180]:
geocoder = Nominatim(user_agent="Toronto_dc")

Toronto = geocoder.geocode({'city': 'Toronto', 'country':'Canada'})
torLat = Toronto.latitude
torLong = Toronto.longitude


In [181]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[torLat, torLong], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfToronto['Latitude'], dfToronto['Longitude'], dfToronto['Borough'], dfToronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [182]:
VERSION = '20180605' # Foursquare API version
radius = 500
limit = 100

In [183]:
# @hidden_cell
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        print(categories_list)
        return categories_list[0]['name']

The following cell contains the code to grab toronto venues. As sometimes it outputs an error, once it was run without problems, I saved the got Toronto venues in a .csv file that I will read for future usage of this notebook. So I ave commented out all its and further next cell content.

In [184]:
'''
def getNearbyVenues(names, latitudes, longitudes):
    numb = 1
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(numb, ' ', name)
        numb +=1    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?oauth_token={}&ll={},{}&v={}&radius={}&limit={}'.format(
            myToken,  
            lat, 
            lng, 
            VERSION,
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        if numb == 2:
            nearby_venues.columns = ['Neighborhood', 
                                      'Neighborhood Latitude', 
                                      'Neighborhood Longitude', 
                                      'Venue', 
                                      'Venue Latitude', 
                                      'Venue Longitude', 
                                      'Venue Category']
        nearby_venues.to_csv("toronto_venues.csv")
    return(nearby_venues)
'''

'\ndef getNearbyVenues(names, latitudes, longitudes):\n    numb = 1\n    venues_list=[]\n    for name, lat, lng in zip(names, latitudes, longitudes):\n        print(numb, \' \', name)\n        numb +=1    \n        # create the API request URL\n        url = \'https://api.foursquare.com/v2/venues/explore?oauth_token={}&ll={},{}&v={}&radius={}&limit={}\'.format(\n            myToken,  \n            lat, \n            lng, \n            VERSION,\n            radius, \n            limit)\n            \n        # make the GET request\n        results = requests.get(url).json()[\'response\'][\'groups\'][0][\'items\']\n        \n        # return only relevant information for each nearby venue\n        venues_list.append([(\n            name, \n            lat, \n            lng, \n            v[\'venue\'][\'name\'], \n            v[\'venue\'][\'location\'][\'lat\'], \n            v[\'venue\'][\'location\'][\'lng\'],  \n            v[\'venue\'][\'categories\'][0][\'name\']) for v in results])

In [185]:
'''
toronto_venues = getNearbyVenues(names=dfToronto['Neighborhood'],
                                   latitudes=dfToronto['Latitude'],
                                   longitudes=dfToronto['Longitude'],
                                  )
'''

"\ntoronto_venues = getNearbyVenues(names=dfToronto['Neighborhood'],\n                                   latitudes=dfToronto['Latitude'],\n                                   longitudes=dfToronto['Longitude'],\n                                  )\n"

In [186]:
# The code was removed by Watson Studio for sharing.

Let's import the saved .csv file with Toronto venues into a dataframe called toronto_venues

In [187]:
tor_ven = client_de7790dfb63141b0ae0ffdfc73f2f446.get_object(Bucket='courseracapstone-donotdelete-pr-cvzsktymx4cuiu',Key='toronto_venues.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(tor_ven, "__iter__"): tor_ven.__iter__ = types.MethodType( __iter__, tor_ven )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
toronto_venues = pd.read_csv(tor_ven)
toronto_venues.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,NT Home Service Inc.,43.806411,-79.197736,Home Service
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
4,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service


Let's check the shape of the new dataframe

In [188]:
toronto_venues.shape

(3102, 7)

Now, in order to have a first undertanding of the Neighborhood venues, let's output the dataframe where, for every Neighborhood, the total number of got venues is printed

In [189]:
toronto_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,7,7,7,7,7,7
"Alderwood, Long Branch",11,11,11,11,11,11
"Bathurst Manor, Wilson Heights, Downsview North",32,32,32,32,32,32
Bayview Village,5,5,5,5,5,5
"Bedford Park, Lawrence Manor East",55,55,55,55,55,55
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",34,34,34,34,34,34
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",23,23,23,23,23,23


We have set the limit of returned venues number at 100. We see that some Neighborhoods have as few as only 1 venue, other reach the limit of 100 and so maybe they could have much more.

Another interesting observation is that some Neighborhood "spread" across multiple Postal Codes, so, when we group them, we have a dataframe of a smaller size (less rows) than the original one. Let's see how many distintc Neighborhood we have found in Toronto.

In [190]:
toronto_venues.groupby("Neighborhood").count().shape

(98, 6)

Ok, we have 98 distinct Neighborhoods in Toronto that have at least 1 venue

Let's see now how many venues categories types are in the dataframe

In [191]:
print("There are ", len(toronto_venues['Venue Category'].unique()), " distinct venue categories in Toronto")

There are  321  distinct venue categories in Toronto


There are 321 distinct venue categories. So let's encode our dataframe in order to use it with machine learning algorithm

In [192]:
toronto_venues_dum = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
print(toronto_venues_dum.shape)
toronto_venues_dum.head()

(3102, 321)


,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Quad,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating R

Let's add to our new dataframe the column with the names of the Neighborhoods

In [193]:
toronto_venues_dum['Neighborhood'] = toronto_venues['Neighborhood']
toronto_venues_dum.head()

,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Quad,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating R

Let's move the Neighborhood column at the first position

In [194]:
torontoCols = list(toronto_venues_dum.columns)
torontoCols.remove('Neighborhood')
orderedCols = []
orderedCols = ['Neighborhood'] + torontoCols
toronto_venues_dum = toronto_venues_dum[orderedCols]
toronto_venues_dum.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Quad,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating R

Now, let's group the new dataframe in order to have, for every Neighborhood, the mean of every venue in the Neighborhoodand check its shape 

In [195]:
toronto_grouped = toronto_venues_dum.groupby('Neighborhood').mean()
toronto_grouped.reset_index(inplace=True)
toronto_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Quad,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating R

In [196]:
toronto_grouped.shape

(98, 321)

Now, I think another feature to consider is how many venues there are in Neighborhood. This because if you have 10% of one category in a Neighborhood that has 100 venues, it means you have, of that category, much more venues respect another Neighborhood that, for example, has 100% of the same category but a total of only 1 venue. In the first case you have 10 venues of the considered category, in the second only 1. So I have added to the dataframe also the ratio between the total venues number in a Neighborhood and the limit of 100 venues. In this way we have a normalized feature ready to be used in clustering algorithm. So I have added a feature called "Venues density".

In [197]:
toronto_grouped['Venues density'] = (toronto_venues.groupby("Neighborhood").count()['Venue'].values)/100
toronto_grouped['Venues density'] = (toronto_grouped['Venues density']- toronto_grouped['Venues density'].min())/(toronto_grouped['Venues density'].max()- toronto_grouped['Venues density'].min())
toronto_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Quad,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating R

Let's move the new column at the second place

In [198]:
orderedCols = list(toronto_venues_dum.columns)

orderedCols.insert(1, 'Venues density')

toronto_grouped = toronto_grouped[orderedCols]
toronto_grouped.reset_index()
toronto_grouped.head()

,Neighborhood,Venues density,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Quad,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate

Let's summurize the most relevant venues categories in a new dataframe

In [199]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [200]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood', 'Venues density']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
neighborhoods_venues_sorted['Venues density'] = toronto_grouped['Venues density']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,Venues density,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,0.060606,Skating Rink,Hardware Store,Breakfast Spot,Fireworks Store,Latin American Restaurant,Lounge,Clothing Store,Empanada Restaurant,Donut Shop,Drugstore
1,"Alderwood, Long Branch",0.101010,Pizza Place,Pharmacy,Pool,Skating Rink,Dance Studio,Athletics & Sports,Pub,Sandwich Place,Coffee Shop,Gym
2,"Bathurst Manor, Wilson Heights, Downsview North",0.313131,Pharmacy,Ice Cream Shop,Mobile Phone Shop,Spa,Coffee Shop,Bank,Sushi Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,Deli / Bodega
3,Bayview Village,0.040404,Japanese Restaurant,Café,Chinese Restaurant,Bank,Spa,Yoga Studio,Electronics Store,Donut Shop,Drugstore,Dumpling Restaurant
4,"Bedford Park, Lawrence Manor East",0.545455,Spa,Pizza Place,Italian Restaurant,Sandwich Place,Sushi Restaurant,Coffee Shop,Mobile Phone Shop,Juice Bar,Thai Restaurant,Hobby Shop


Ok, here we have an idea of the most common venues in each Neighborhood and how much the Neighborhood is "dense" of activities. We are ready for clustering

### Clustering

First, let's check wich is the best number of cluster. In order to do this let's use the metric silohuette score.  
From scikit webpage:  
> Silhouette analysis can be used to study the separation distance between the resulting clusters. The silhouette plot displays a measure of how close each point in one cluster is to points in the neighboring clusters and thus provides a way to assess parameters like number of clusters visually. This measure has a range of [-1, 1].

> Silhouette coefficients (as these values are referred to as) near +1 indicate that the sample is far away from the neighboring clusters. A value of 0 indicates that the sample is on or very close to the decision boundary between two neighboring clusters and negative values indicate that those samples might have been assigned to the wrong cluster.

In [201]:
X = toronto_grouped.drop(['Neighborhood'], 1)

num_clusters = np.arange(2,10)
results = {}
for k in num_clusters:
    kmeans = KMeans(init="k-means++", n_clusters = k, n_init = 20, random_state=0).fit(X)

    predictions = kmeans.predict(X)
    results[k] = silhouette_score(X, predictions)


best_k = max(results, key = results.get)

print(results,"\nBest k value is: ",best_k)

{2: 0.3695046292746572, 3: 0.17477620154423124, 4: 0.18380969845926104, 5: 0.19479193628764727, 6: 0.2016109735410431, 7: 0.2055290343079372, 8: 0.2148071055881742, 9: 0.2163657769002606} 
Best k value is:  2


It looks that the best value is 2, and the higher k is, the worse the accuracy of clustering is

Now, before going on and applying our model, let's manipulate our dataframes in order to get compatible dataframes

In [202]:
# Let's drop PostalCode column from our original datframe and save it with a new name
dfTorontoMod = dfToronto.drop('PostalCode',1)
dfTorontoMod.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [203]:
#Let's check the shape of the new dataframe
dfTorontoMod.shape

(103, 4)

In order to join our new dataframe to that with the most common venues per Neighborhood in Toronto, we have to get 98 rows. This mismatch is due to the fact that in the original dataframe some Neighborhood spread over multiple Postal Codes.
Let's see which are.

In [204]:
tempdfTorontoMod = dfTorontoMod.groupby('Neighborhood').count()
tempdfTorontoMod

,Borough,Latitude,Longitude
Neighborhood,,,
Agincourt,1,1,1
"Alderwood, Long Branch",1,1,1
"Bathurst Manor, Wilson Heights, Downsview North",1,1,1
Bayview Village,1,1,1
"Bedford Park, Lawrence Manor East",1,1,1
Berczy Park,1,1,1
"Birch Cliff, Cliffside West",1,1,1
"Brockton, Parkdale Village, Exhibition Place",1,1,1
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",1,1,1


In [205]:
tempdfTorontoMod = tempdfTorontoMod[tempdfTorontoMod['Borough'] >1 ]
tempdfTorontoMod

,Borough,Latitude,Longitude
Neighborhood,,,
Don Mills,2,2,2
Downsview,4,4,4


Don Mills and Downsview spread over multiple PostalCodes. Let's check if they also spread over multiple Borough

In [206]:
tempdfTorontoMod = dfTorontoMod[dfTorontoMod['Neighborhood'] == 'Don Mills']
tempdfTorontoMod

,Borough,Neighborhood,Latitude,Longitude
26,North York,Don Mills,43.745906,-79.352188
27,North York,Don Mills,43.725900,-79.340923


Don Mills spread over 2 Postal Codes but it is always in the same Borough. Let's check Downsview also

In [207]:
tempdfTorontoMod = dfTorontoMod[dfTorontoMod['Neighborhood'] == 'Downsview']
tempdfTorontoMod

,Borough,Neighborhood,Latitude,Longitude
30,North York,Downsview,43.737473,-79.464763
31,North York,Downsview,43.739015,-79.506944
32,North York,Downsview,43.728496,-79.495697
33,North York,Downsview,43.761631,-79.520999


Downsview spread over 4 Postal Codes but it is always in the same Borough too.  
So, in order to "clean" our dataframe, let's keep only one row for each of these two Neighborhoods and let's substitute their LAtitude and Longitude values with the average of the values in their rows

In [208]:
dfTorontoMod.loc[26,'Latitude'] = (dfTorontoMod.loc[26,'Latitude'] + dfTorontoMod.loc[27,'Latitude'])/2
dfTorontoMod.loc[26,'Longitude'] = (dfTorontoMod.loc[26,'Longitude'] + dfTorontoMod.loc[27,'Longitude'])/2

dfTorontoMod.loc[30,'Latitude'] = (dfTorontoMod.loc[30,'Latitude'] + dfTorontoMod.loc[31,'Latitude'] + dfTorontoMod.loc[32,'Latitude'] + dfTorontoMod.loc[33,'Latitude'])/4
dfTorontoMod.loc[30,'Longitude'] = (dfTorontoMod.loc[30,'Longitude'] + dfTorontoMod.loc[31,'Longitude'] + dfTorontoMod.loc[32,'Longitude'] + dfTorontoMod.loc[33,'Longitude'])/4

dfTorontoMod.drop([27,31,32,33], inplace=True)
dfTorontoMod.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


And let's check the shape of the dataframe

In [209]:
print(dfTorontoMod.shape)
dfTorontoMod.head()

(99, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


Ops! We have still one row more than that we expected and the new dataframe has still its original ordere based upon PostalCode. So let's reorder it and compare the Neighborhood names containd in this new dataframe and those in that one (neighborhoods_venues_sorted) we will join this one to 

In [210]:
dfTorontoMod.sort_values('Neighborhood', inplace=True)
dfTorontoMod = dfTorontoMod.reset_index(drop=True)
dfTorontoMod.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,Agincourt,43.794200,-79.262029
1,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
3,North York,Bayview Village,43.786947,-79.385975
4,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750


In [211]:
i = 0
for neigh1, neigh2 in zip(dfTorontoMod['Neighborhood'], neighborhoods_venues_sorted["Neighborhood"]):
    if not (neigh1 == neigh2):
        print(i,": ",neigh1," - ", neigh2)
    i += 1

86 :  Upper Rouge  -  Victoria Village
87 :  Victoria Village  -  West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
88 :  West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale  -  Westmount
89 :  Westmount  -  Weston
90 :  Weston  -  Wexford, Maryvale
91 :  Wexford, Maryvale  -  Willowdale, Newtonbrook
92 :  Willowdale, Newtonbrook  -  Willowdale, Willowdale East
93 :  Willowdale, Willowdale East  -  Willowdale, Willowdale West
94 :  Willowdale, Willowdale West  -  Woburn
95 :  Woburn  -  Woodbine Heights
96 :  Woodbine Heights  -  York Mills West
97 :  York Mills West  -  York Mills, Silver Hills


At row number 86 we have a different value: dfTorontoMod has "Upper Rouge", while neighborhoods_venues_sorted has not at all that value. This is probably due to the fact that maybe Upper Rouge had no venues, so it is not present in the neighborhoods_venues_sorted dataframe. Let's fix that!

In [212]:
tempdfTorontoMod = dfTorontoMod[dfTorontoMod['Neighborhood']=='Upper Rouge']
tempdfTorontoMod

,Borough,Neighborhood,Latitude,Longitude
86,Scarborough,Upper Rouge,43.836125,-79.205636


In [213]:
dfTorontoMod.drop([86], inplace=True)
dfTorontoMod = dfTorontoMod.reset_index(drop=True)
print(dfTorontoMod.shape)
dfTorontoMod.head()

(98, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,Agincourt,43.794200,-79.262029
1,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259
3,North York,Bayview Village,43.786947,-79.385975
4,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750


Let's do last check to see if the two Neighborhood series in the two dataframe are exactly equal

In [214]:
index1 = neighborhoods_venues_sorted["Neighborhood"]
index2 = dfTorontoMod["Neighborhood"]

if (index1.all() == index2.all()):
    print("Dataframes are compatible!")
else:
    print("Something wrong! Please check!")

Dataframes are compatible!


Perfect, we can use those dataframe for the next part

Ok, let's cluster the Neighborhood with K-Means algorithm and let's plot them

In [215]:
X = toronto_grouped.drop(['Neighborhood'], 1)
kmeans = KMeans(init="k-means++", n_clusters = best_k, n_init = 20, random_state=0).fit(X)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_
toronto_merged = dfTorontoMod

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,Venues density,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,Agincourt,43.794200,-79.262029,1,0.060606,Skating Rink,Hardware Store,Breakfast Spot,Fireworks Store,Latin American Restaurant,Lounge,Clothing Store,Empanada Restaurant,Donut Shop,Drugstore
1,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484,1,0.101010,Pizza Place,Pharmacy,Pool,Skating Rink,Dance Studio,Athletics & Sports,Pub,Sandwich Place,Coffee Shop,Gym
2,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,1,0.313131,Pharmacy,Ice Cream Shop,Mobile Phone Shop,Spa,Coffee Shop,Bank,Sushi Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,Deli / Bodega
3,North York,Bayview Village,43.786947,-79.385975,1,0.040404,Japanese Restaurant,Café,Chinese Restaurant,Bank,Spa,Yoga Studio,Electronics Store,Donut Shop,Drugstore,Dumpling Restaurant
4,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,0,0.545455,Spa,Pizza Place,Italian Restaurant,Sandwich Place,Sushi Restaurant,Coffee Shop,Mobile Phone Shop,Juice Bar,Thai Restaurant,Hobby Shop


In [216]:
# create map
map_clusters = folium.Map(location=[torLat, torLong], zoom_start=11)

# set color scheme for the clusters
x = np.arange(best_k)
ys = [i + x + (i*x)**2 for i in range(best_k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's look inside the two found clusters

### Cluster 0

In [217]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Venues density,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Bedford Park, Lawrence Manor East",0.545455,Spa,Pizza Place,Italian Restaurant,Sandwich Place,Sushi Restaurant,Coffee Shop,Mobile Phone Shop,Juice Bar,Thai Restaurant,Hobby Shop
5,Berczy Park,1.000000,Coffee Shop,Bakery,Cocktail Bar,Pub,Italian Restaurant,Restaurant,Moroccan Restaurant,Cheese Shop,Bar,Bistro
13,Central Bay Street,1.000000,Coffee Shop,Café,Sandwich Place,Sushi Restaurant,Italian Restaurant,Ice Cream Shop,Thai Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Indian Restaurant
15,Church and Wellesley,1.000000,Coffee Shop,Gay Bar,Sushi Restaurant,Japanese Restaurant,Dessert Shop,Restaurant,Burger Joint,Yoga Studio,Bubble Tea Shop,Ramen Restaurant
18,"Commerce Court, Victoria Hotel",1.000000,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,Beer Bar
19,Davisville,0.505051,Dessert Shop,Sandwich Place,Pizza Place,Café,Gas Station,Coffee Shop,Gym,Italian Restaurant,Thai Restaurant,Sushi Restaurant
28,"Fairview, Henry Farm, Oriole",1.000000,Clothing Store,Coffee Shop,Women's Store,Fast Food Restaurant,Cosmetics Shop,Shoe Store,Sporting Goods Shop,Bank,Restaurant,Kids Store
29,"First Canadian Place, Underground city",1.000000,Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,Deli / Bodega,Seafood Restaurant,Salad Place,Steakhouse
31,"Garden District, Ryerson",1.000000,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Middle Eastern Restaurant,Italian Restaurant,Cosmetics Shop,Japanese Restaurant,Electronics Store,Fast Food Restaurant
35,"Harbourfront East, Union Station, Toronto Islands",1.000000,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Brewery,Italian Restaurant,Scenic Lookout,Fried Chicken Joint,History Museum


Cluster 0 is that with an higher density of venues. It is in the central zone of Toronto and the venue categories are most likely Restaurant, Coffee shops, Boutiques

### Cluster 1

In [218]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Venues density,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,0.060606,Skating Rink,Hardware Store,Breakfast Spot,Fireworks Store,Latin American Restaurant,Lounge,Clothing Store,Empanada Restaurant,Donut Shop,Drugstore
1,"Alderwood, Long Branch",0.101010,Pizza Place,Pharmacy,Pool,Skating Rink,Dance Studio,Athletics & Sports,Pub,Sandwich Place,Coffee Shop,Gym
2,"Bathurst Manor, Wilson Heights, Downsview North",0.313131,Pharmacy,Ice Cream Shop,Mobile Phone Shop,Spa,Coffee Shop,Bank,Sushi Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,Deli / Bodega
3,Bayview Village,0.040404,Japanese Restaurant,Café,Chinese Restaurant,Bank,Spa,Yoga Studio,Electronics Store,Donut Shop,Drugstore,Dumpling Restaurant
6,"Birch Cliff, Cliffside West",0.040404,Café,College Stadium,Skating Rink,General Entertainment,Farm,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore
7,"Brockton, Parkdale Village, Exhibition Place",0.333333,Café,Performing Arts Venue,Breakfast Spot,Nightclub,Sandwich Place,Coffee Shop,Stadium,Sports Bar,Sporting Goods Shop,Smoke Shop
8,"Business reply mail Processing Centre, South C...",0.222222,Light Rail Station,Gym / Fitness Center,Fast Food Restaurant,Yoga Studio,Martial Arts Dojo,Auto Workshop,Spa,Smoke Shop,Skate Park,Brewery
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.181818,Airport Service,Airport Terminal,Rental Car Location,Airport Lounge,Boat or Ferry,Plane,Coffee Shop,Sculpture Garden,Boutique,Airport Gate
10,Caledonia-Fairbanks,0.040404,Park,Women's Store,Spa,Miscellaneous Shop,Yoga Studio,Empanada Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
11,Canada Post Gateway Processing Centre,0.111111,Coffee Shop,Hotel,Mediterranean Restaurant,Fried Chicken Joint,Middle Eastern Restaurant,Burrito Place,Gym,American Restaurant,Intersection,Sandwich Place


Cluster 1 has a lower venues density, typpical of non central Neighborhood and here you can fin more often other activities than Restaurant or Coffe Shop: Business service, Discount, Pharmacy, Hotel, etc.. begin to appear between the most common venue categories.

## Conclusions

We have found that the best number of cluster was only 2. This is most likely because we introduced the "Venues density" feature, that has biased the result toward the number of venues. Anyway, this seems to be meaningful as:  
* the clustering has diveded the city in a central zone, where you can find lot of venues and in a non-central one
* the 2 zones venues are somehow different: in the first one you can find venues more dedicated to living outside, in the second more "everyday needings" venues

### **This concludes the last part of the first Notebook for final assignment.**